In [1]:
import pandas as pd
import spacy 
from nltk.classify import NaiveBayesClassifier
import text_processor as tp

In [9]:
sp = spacy.load('en', parse=True, tag=True, entity=True)

In [10]:
def lemmatize_text(text):
    """
    Lemmatize the words using spacy
    Note: Pronouns are excluded (eg: he, she, my, it etc...)
    """
    try:
        text = sp(text)
        text = ' '.join([word.lemma_ if word.lemma_ != '-PRON-' else word.text for word in text])
    except TypeError:
        pass
    return text

In [30]:
text = "Everything is great except that i can't read external pdfs properly. Either it zooms too much of too little. For kindle books it provides great experience. Battery life is awesome!"

In [31]:
doc = sp(tp.clean_text(text))

In [32]:
for token in doc:
    print(token, token.pos_, token.lemma_)

everything PRON everything
great ADJ great
except SCONJ except
read VERB read
external ADJ external
pdfs NOUN pdfs
properly ADV properly
either CCONJ either
zoom VERB zoom
much ADJ much
little ADJ little
kindle NOUN kindle
book NOUN book
provide VERB provide
great ADJ great
experience NOUN experience
battery NOUN battery
life NOUN life
awesome ADJ awesome


In [33]:
for token in doc:
    if token.pos_ == 'NOUN' or token.pos_ == 'ADJ':
        print(token, token.pos_)

great ADJ
external ADJ
pdfs NOUN
much ADJ
little ADJ
kindle NOUN
book NOUN
great ADJ
experience NOUN
battery NOUN
life NOUN
awesome ADJ


In [2]:
df = pd.read_csv('datasets/AllProductReviews.csv')

In [4]:
df.count()

ReviewTitle    14337
ReviewBody     14337
ReviewStar     14337
Product        14337
dtype: int64

In [5]:
df.groupby(df['ReviewStar']).count()

,ReviewTitle,ReviewBody,Product
ReviewStar,,,
1,2493,2493,2493
2,939,939,939
3,1503,1503,1503
4,3189,3189,3189
5,6213,6213,6213


In [6]:
df1 = df[df['ReviewStar'] < 3]

In [7]:
df1.head()

,ReviewTitle,ReviewBody,ReviewStar,Product
1,Unreliable earphones with high cost\n,"This earphones are unreliable, i bought it be...",1,boAt Rockerz 255
3,stopped working in just 14 days\n,Its sound quality is adorable. overall it was ...,1,boAt Rockerz 255
5,Charging port not working\n,"After 11 days, the charging port isn't working...",1,boAt Rockerz 255
9,Very very bad Durabity\n,The product durability is 1 month..I just lost...,1,boAt Rockerz 255
11,Disappointed\n,What on earth is the use of buying such produc...,1,boAt Rockerz 255


In [12]:
df1 = df1.drop(['ReviewTitle', 'Product'], axis=1)

In [13]:
df1.head()

,ReviewBody,ReviewStar
1,"This earphones are unreliable, i bought it be...",1
3,Its sound quality is adorable. overall it was ...,1
5,"After 11 days, the charging port isn't working...",1
9,The product durability is 1 month..I just lost...,1
11,What on earth is the use of buying such produc...,1


In [14]:
df1 = df1.rename(columns={'ReviewBody':'content','ReviewStar':'rating'})

In [15]:
df1.head()

,content,rating
1,"This earphones are unreliable, i bought it be...",1
3,Its sound quality is adorable. overall it was ...,1
5,"After 11 days, the charging port isn't working...",1
9,The product durability is 1 month..I just lost...,1
11,What on earth is the use of buying such produc...,1


In [16]:
df1.count()

content    3432
rating     3432
dtype: int64

In [ ]:
d['sentiment'] = filtered_df['reviews.rating'].map(sentiment_dict)